## Causal Inference

In [1]:
import pandas as pd


In [2]:
df_T1 = pd.read_json('/workspaces/chat-gpt-failures/data/causal_prompting_pd_T2.json')

In [3]:
df_T1.head()

,z_n_ast_errors,z_ast_levels,z_n_whitespaces,z_complexity,z_nloc,z_token_counts,z_n_ast_nodes,y_levenshtein,t_binary_treatment
0,0,11,46,1,9,47,185,527,False
1,0,15,141,7,38,233,493,1826,False
2,0,10,95,1,45,182,308,1228,False
3,1,13,61,1,37,178,314,1544,False
4,0,12,35,4,14,80,141,337,False


## I. Create a causal model from the data and given graph.

In [4]:
from dowhy import CausalModel

In [29]:
model = CausalModel(
        data = df_T1,
        treatment = ['t_binary_treatment'],
        outcome = ['y_levenshtein'],
        common_causes = ['z_n_ast_errors','z_ast_levels','z_n_whitespaces','z_complexity','z_nloc','z_token_counts','z_n_ast_nodes']
        #common_causes= ['z_nloc']
    )

## II. Identify causal effect and return target estimands.

In [31]:
identified_estimand = model.identify_effect()

# III. Estimate the target estimand using a statistical method.


In [32]:
causal_estimate_strat = model.estimate_effect(identified_estimand,method_name="backdoor.propensity_score_stratification")

propensity_score_stratification


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
causal_estimate_matching = model.estimate_effect(
        identified_estimand,
        method_name="backdoor.propensity_score_matching")

propensity_score_matching


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
          d                                                                   
─────────────────────(E[y_levenshtein|z_n_ast_errors,z_token_counts,z_n_whites
d[t_binary_treatment]                                                         

                                                      
paces,z_complexity,z_n_ast_nodes,z_ast_levels,z_nloc])
                                                      
Estimand assumption 1, Unconfoundedness: If U→{t_binary_treatment} and U→y_levenshtein then P(y_levenshtein|t_binary_treatment,z_n_ast_errors,z_token_counts,z_n_whitespaces,z_complexity,z_n_ast_nodes,z_ast_levels,z_nloc,U) = P(y_levenshtein|t_binary_treatment,z_n_ast_errors,z_token_counts,z_n_whitespaces,z_complexity,z_n_ast_nodes,z_ast_levels,z_nloc)

## Realized estimand
b: y_levenshtein~t_binary_treatment+z_n_ast_errors+z_token_counts+z_n_whitesp

In [9]:
es_placebo = causal_model.refute_estimate(
                identified_estimand,
                estimate,
                method_name="placebo_treatment_refuter",
                placebo_type="permute"
        )

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

# IV. Refute the obtained estimate using multiple robustness checks.


In [11]:
print(es_placebo)

Refute: Use a Placebo Treatment
Estimated effect:-438.07140294223177
New effect:0.34772156440617186
p value:0.98



In [12]:

refute_results = causal_model.refute_estimate(identified_estimand, estimate,
                                       method_name="random_common_cause")

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [14]:
print(refute_results)

Refute: Add a random common cause
Estimated effect:-438.07140294223177
New effect:102.55903480444924
p value:0.0



In [18]:
res_subset=causal_model.refute_estimate(identified_estimand, estimate,
        method_name="data_subset_refuter", subset_fraction=0.9)
print(res_subset)

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

Refute: Use a subset of data
Estimated effect:-438.07140294223177
New effect:101.6067171616504
p value:0.0



## Library Test

In [21]:
import dowhy.datasets 

In [22]:
df_T1.shape

In [24]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5, 
        num_instruments = 0,
        num_treatments=1,
        num_samples=5574,
        treatment_is_binary=True,
        outcome_is_binary=False)
df = data["df"]
df

,W0,W1,W2,W3,W4,v0,y
0,0.941101,-0.301966,-0.535682,-0.965963,-1.796073,False,-9.524400
1,-3.236478,2.193964,0.083692,-1.513227,1.448192,True,5.881971
2,-0.963139,-1.507044,-2.078690,-1.703016,-1.388056,False,-30.613549
3,-1.382753,-2.106809,-2.662127,-0.986787,0.198368,False,-29.213463
4,0.496856,1.076723,-0.331041,-1.548117,-0.169493,False,-0.416004
...,...,...,...,...,...,...,...
5569,0.178459,0.583549,1.439157,-0.704502,-0.441188,True,16.003207
5570,0.378616,0.274978,-0.955388,-2.081811,0.072679,False,-7.743903
5571,-1.821017,0.346252,-3.560790,0.664515,-1.107096,False,-25.758491
5572,-1.291284,-0.944791,-0.354400,0.536646,1.767576,True,7.415752


In [25]:
causal_model = CausalModel(
        data = df,
        treatment = ['v0'],
        outcome = ['y'],
        common_causes = ['W0','W1','W2','W3','W4']
        #common_causes= ['z_nloc']
    )

In [26]:
identified_estimand = causal_model.identify_effect()
estimate = causal_model.estimate_effect(
        identified_estimand,
        method_name="backdoor.propensity_score_matching")

propensity_score_matching


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                       
─────(E[y|W2,W0,W1,W3,W4])
d[v₀]                     
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W2,W0,W1,W3,W4,U) = P(y|v0,W2,W0,W1,W3,W4)

## Realized estimand
b: y~v0+W2+W0+W1+W3+W4
Target units: ate

## Estimate
Mean value: 12.621883737314798



In [28]:
es_placebo = causal_model.refute_estimate(
                identified_estimand,
                estimate,
                method_name="placebo_treatment_refuter",
                placebo_type="permute"
        )

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [33]:
print(es_placebo)

Refute: Use a Placebo Treatment
Estimated effect:12.621883737314798
New effect:-0.011348136102024025
p value:1.0

